# Coursework 1 : Movie Recommendation from Text


In [1]:
from collections import Counter
import math
import pandas as pd
import pickle

Load documents from file and process them

In [2]:
movies_meta=pd.read_csv("data/movie.metadata.tsv", 
                         sep='\t', header=None, usecols=[0,1,2,3,8], 
                         names=['wID', 'fID', 'title', 'data', 'genres' ])

movies_plot=pd.read_csv("data/plot_summaries.txt", 
                         sep='\t', header=None, usecols=[0,1], 
                         names=['wID', 'plot'])

movies_merged = pd.merge(movies_meta, movies_plot, on='wID', how='inner')
    
print("Retrieved {} ".format(len(movies_merged)))

with open("data/vocabulary.pk","rb") as pickle_in:
    vocabulary = pickle.load(pickle_in)

with open("data/tf_idf.pk","rb") as pickle_in:
    tf_idf = pickle.load(pickle_in)
    
# print stats

Retrieved 42204 


Compute similarity between target and other docs

In [5]:
def analyze(target_row):  
  tfidf =  tf_idf[target_row['wID']]
  num_words = 20
  sorted_tfidf = [
    w  for (w, _) in sorted(tfidf.items(), key=lambda kv: kv[1], reverse=True)
  ]
  print(target_row['title'])
  print("Higher TF-IDF: {}".format(sorted_tfidf[:num_words]))


def cosine_dic(dic1,dic2):
    numerator = 0
    dena = 0
    for key1,val1 in dic1.items():
        numerator += val1*dic2.get(key1,0.0)
        dena += val1*val1
    denb = 0
    for val2 in dic2.values():
        denb += val2*val2
    return numerator/math.sqrt(dena*denb)    
    
def similar(target_1, target_2):
    tfidf1 = tf_idf[target_1['wID']]
    tfidf2 = tf_idf[target_2['wID']]
    return cosine_dic(tfidf1, tfidf2)
    
    
    


target_1 = movies_merged.loc[movies_merged['title'] == "Mary Poppins"].iloc[0]

#target_2 = movies_merged.loc[movies_merged['title'] == "Grease"].values[0]

#analyze(target_1)
#analyze(target_2)

countp = 0
for idx, movie in movies_merged.iterrows():
    scr = similar(target_1, movie)
    if scr > 0.15:
        print("{} {}".format(movie['title'], scr)) 
    countp+=1
    if countp > 1099:
        break
print('done')

# analyze_page("Moana")
# analyze_page("The Incredibles 2004")
# analyze_page("Monsters, Inc")

Mary Poppins 1.0
Sabata 0.15344422092958465
Afterglow 0.1703957922421328
Sesame Street presents Follow That Bird 0.15247784609524465
done
